In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np

import datajoint as dj
import spyglass

from spyglass.common import (
    Session,
    IntervalList,
    LabMember,
    LabTeam,
    Raw,
    Session,
    Nwbfile,
    TaskEpoch,
    Electrode,
    ElectrodeGroup,
    LFP,
    LFPSelection,
    LFPBand,
    LFPBandSelection,
)
from spyglass.common.common_position import RawPosition, IntervalPositionInfo

os.chdir("/home/sambray/Documents/MS_analysis_Jen/")
from ms_dio_event import DioEvents, ProcessedDioEvents
from ms_task_identification import TaskIdentification
from ms_interval import EpochIntervalListName
from metadata_helpers import get_ms_nwbf_epoch_keys
from vector_helpers import unpack_single_element
from populate_position_tables import populate_position_info_table
from plot_helpers import format_ax
from stats_helpers import average_confidence_interval
from point_process_helpers import event_times_in_intervals_bool
from ms_dio_event import *
from ms_task_performance import *

os.chdir("/home/sambray/Documents/MS_analysis_samsplaying/")
from ms_opto_stim_protocol import OptoStimProtocol
from IPython.core.display import display, HTML

display(
    HTML("<style>.container { width:100% !important; }</style>")
)  # set cells to be wide

[2023-10-25 14:54:30,483][INFO]: Connecting sambray@lmf-db.cin.ucsf.edu:3306
[2023-10-25 14:54:30,517][INFO]: Connected sambray@lmf-db.cin.ucsf.edu:3306
/tmp/ipykernel_1349663/3770185089.py:29: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Define Sessions



In [ ]:
animal_names = [
    "Frodo",
    "Bilbo",
    "Winnie",
]  # ['Yoshi',"Winnie","Wallie","Banner","Totoro","Olive","Odins","Dan"]
nwb_file_names = []

[
    nwb_file_names.extend((Session() & {"subject_id": animal}).fetch("nwb_file_name"))
    for animal in animal_names
]
# nwb_file_names = ['Frodo20230808_.nwb']

[None, None, None]

# Populate!

In [ ]:
skipped_sessions
TaskIdentification() & {"nwb_file_name": "Frodo20230814_.nwb"}
# key

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),contingency,task_environment
Frodo20230814_.nwb,1,Sleep,sleepbox
Frodo20230814_.nwb,2,W-Track,W-track
Frodo20230814_.nwb,3,Sleep,sleepbox
Frodo20230814_.nwb,4,W-Track,W-track
Frodo20230814_.nwb,5,Sleep,sleepbox
Frodo20230814_.nwb,6,W-Track,W-track
Frodo20230814_.nwb,7,Sleep,sleepbox
Frodo20230814_.nwb,8,W-Track,W-track
Frodo20230814_.nwb,9,Sleep,sleepbox
Frodo20230814_.nwb,10,W-Track,W-track


In [ ]:
import warnings

warnings.filterwarnings("ignore")

bad_keys = [{"nwb_file_name": "Bilbo20230725_.nwb", "epoch": 8}]

# nwb_file_names = ["Yoshi20220510_.nwb"]
analysis_path = "/stelmo/nwb/analysis/"
skipped_sessions = []
for nwb_file_name in nwb_file_names:
    print(f"on {nwb_file_name}...")
    # skip session if you don't have write privledges
    if os.path.isdir(analysis_path + nwb_file_name.split("_")[0]) and (
        not os.access(analysis_path + nwb_file_name.split("_")[0], os.W_OK)
    ):
        skipped_sessions.append(nwb_file_name)
        continue
    key = {"nwb_file_name": nwb_file_name}
    if not TaskIdentification & key:
        TaskIdentification.populate(key)
    # epochs = np.append((TaskIdentification & {**key, **{"contingency": "wtrack"}}).fetch("epoch"),(TaskIdentification & {**key, **{"contingency": "lineartrack"}}).fetch("epoch"))
    epochs = []
    valid_contingency = [
        "Wtrack",
        "W-track",
        "wtrack",
        "w-track",
        "lineartrack",
        "linear-track",
        "linear-track",
        "Lineartrack",
    ]
    for contingency in valid_contingency:
        epochs.extend(
            (TaskIdentification & {**key, **{"contingency": contingency}}).fetch(
                "epoch"
            )
        )

    epochs = np.append(
        epochs,
        (TaskIdentification & {**key, **{"contingency": "Wtrack"}}).fetch("epoch"),
    )
    for epoch in epochs:
        print(f"on epoch {epoch}...")
        key.update({"epoch": epoch})
        if key in bad_keys:
            continue
        if not (EpochIntervalListName & key):
            EpochIntervalListName.populate(key)
            print("populated epoch", key)
        if not DioEvents & key:
            DioEvents.populate(key)
        if not ProcessedDioEvents & key:
            ProcessedDioEvents.populate(key)
        if not AlternationTaskPerformance & key:
            AlternationTaskPerformance.populate(key)

on Frodo20230808_.nwb...
on epoch 2...
on epoch 4...
on epoch 6...
on epoch 8...
on epoch 10...
on epoch 12...
on epoch 2...
on epoch 4...
on epoch 6...
on epoch 8...
on epoch 10...
on epoch 12...
on Frodo20230809_.nwb...
on epoch 2...
on epoch 4...
on epoch 6...
on epoch 8...
on epoch 10...
on epoch 12...
on epoch 2...
on epoch 4...
on epoch 6...
on epoch 8...
on epoch 10...
on epoch 12...
on Frodo20230810_.nwb...
on epoch 2...
on epoch 4...
on epoch 6...
on epoch 8...
on epoch 10...
on epoch 12...
on epoch 14...
on epoch 16...
on epoch 18...
on epoch 2...
on epoch 4...
on epoch 6...
on epoch 8...
on epoch 10...
on epoch 12...
on epoch 14...
on epoch 16...
on epoch 18...
on Frodo20230811_.nwb...
on epoch 2...
on epoch 4...
on epoch 6...
populated epoch {'nwb_file_name': 'Frodo20230811_.nwb', 'epoch': 6}
on epoch 8...
on epoch 10...
on epoch 12...
on epoch 14...
on epoch 16...
on epoch 18...
populated epoch {'nwb_file_name': 'Frodo20230811_.nwb', 'epoch': 18}
on epoch 20...
on epoch 2.

In [ ]:
key = {"nwb_file_name": "Frodo20230812_.nwb", "epoch": 4}
TaskIdentification.populate(key)
TaskIdentification & key
TaskEpoch & key

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),task_name,camera_name,interval_list_name descriptive name of this interval list,task_environment the environment the animal was in,camera_names list of keys corresponding to entry in CameraDevice


In [ ]:
set(TaskIdentification().fetch("contingency"))
EpochIntervalListName.populate(key)
key

KeyboardInterrupt: 

In [26]:
key

{'nwb_file_name': 'Frodo20230811_.nwb', 'epoch': 6}

In [ ]:
#standardize contingency names
for key in TaskIdentification.fetch("KEY"):
    contingency = (TaskIdentification & key).fetch1("contingency")
    if contingency in


In [ ]:
# EpochIntervalListName() & {"nwb_file_name":"Frodo20230809_.nwb"}
# DioEvents & {"nwb_file_name":"Frodo20230809_.nwb"}
TaskIdentification & {"nwb_file_name": "Frodo20230812_.nwb"}  # 'epoch': 14}
# print((EpochIntervalListName * Session) & {'subject_id':'Frodo'})
# ((EpochIntervalListName * Session) & {'subject_id':'Frodo'}).fetch('nwb_file_name')
TaskIdentification

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),contingency,task_environment


In [ ]:
# (IntervalList& {"nwb_file_name":"Frodo20230809_.nwb"}).fetch("interval_list_name")
TaskEpoch()  # & {"nwb_file_name":"Frodo20230809_.nwb"}

nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),task_name,camera_name,interval_list_name descriptive name of this interval list,task_environment the environment the animal was in,camera_names list of keys corresponding to entry in CameraDevice
aj8020230628_.nwb,2,Lineartrack,None,02_lineartrack,Lineartrack,=BLOB=
aj8020230628_.nwb,4,Lineartrack,None,04_lineartrack,Lineartrack,=BLOB=
aj8020230628_.nwb,6,Lineartrack,None,06_lineartrack,Lineartrack,=BLOB=
aj8420230703_.nwb,1,Lineartrack,None,01_lineartrack,Lineartrack,=BLOB=
aj8620230628_.nwb,2,Lineartrack,None,02_lineartrack,Lineartrack,=BLOB=
aj8620230628_.nwb,4,Lineartrack,None,04_lineartrack,Lineartrack,=BLOB=
aj8620230628_.nwb,6,Lineartrack,None,06_lineartrack,Lineartrack,=BLOB=
arthur20220314_.nwb,1,Sleep,MEC_sleep_camera,01_s1,none,=BLOB=
arthur20220314_.nwb,2,Content feedback,MEC_run_camera,02_r1,none,=BLOB=
arthur20220314_.nwb,3,Sleep,MEC_sleep_camera,03_s2,none,=BLOB=
